# MCP Calculator with Resources Demo

This notebook demonstrates how to use MCP resources to maintain state across tool calls. Our enhanced calculator server now:
- Maintains a history of all calculations
- Provides statistics about the session
- Allows retrieval of specific calculations
- Generates summaries of calculation activity

## What are MCP Resources?
Resources in MCP are read-only data sources that clients can access. Unlike tools (which perform actions), resources provide access to server state and data.

## Setup and Imports

First, let's import the necessary libraries and set up our connection.

In [1]:
import asyncio
import json
from mcp import ClientSession
from mcp.client.sse import sse_client
from mistralai import Mistral
import sys
from datetime import datetime

## Configure Mistral AI Client

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()
# Set your Mistral API key
if 'MISTRAL_KEY' not in os.environ:
    raise ValueError("MISTRAL_KEY environment variable is not set.")
mistral = Mistral(api_key=os.getenv('MISTRAL_KEY'))

## Connect to the Enhanced Calculator Server

Make sure your server is running with:
```bash
python calculator_server_with_resources.py
```

In [3]:
# Connect and discover available tools and resources
server_url = "http://localhost:9321/sse"

async with sse_client(server_url) as (read, write):
    async with ClientSession(read, write) as session:
        await session.initialize()
        
        # List available tools
        tools_result = await session.list_tools()
        print(f"📊 Calculator ready! Found {len(tools_result.tools)} tools:")
        for tool in tools_result.tools:
            print(f"  - {tool.name}: {tool.description}")
        
        # List available resources
        resources_result = await session.list_resources()
        print(f"\n📁 Found {len(resources_result.resources)} resources:")
        for resource in resources_result.resources:
            print(f"  - {resource.uri}: {resource.description}")

📊 Calculator ready! Found 6 tools:
  - add: Add two numbers together and save to history
  - subtract: Subtract b from a and save to history
  - multiply: Multiply two numbers and save to history
  - divide: Divide a by b and save to history
  - clear_history: Clear the calculation history
  - get_last_result: Get the last calculation result

📁 Found 3 resources:
  - calculation://history: Get the complete calculation history as JSON
  - calculation://statistics: Get calculation statistics
  - calculation://summary: Get a summary of all calculations


## Clear History (Start Fresh)

Let's start with a clean slate by clearing any existing history.

In [4]:
async with sse_client(server_url) as (read, write):
    async with ClientSession(read, write) as session:
        await session.initialize()
        
        # Clear history
        result = await session.call_tool("clear_history", {})
        print(result.content[0].text)

History cleared successfully


## Perform Multiple Calculations

Let's perform several calculations to build up our history.

In [5]:
mistral_tools = []
for tool in tools_result.tools:
    tool_def = {
        "type": "function",  # Required wrapper
        "function": {        # Function definition
            "name": tool.name,
            "description": tool.description,
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {
                        "type": "number",
                        "description": "First number"
                    },
                    "b": {
                        "type": "number",
                        "description": "Second number"
                    }
                },
                "required": ["a", "b"]
            }
        }
    }
    mistral_tools.append(tool_def)

In [6]:
mistral_tools

[{'type': 'function',
  'function': {'name': 'add',
   'description': 'Add two numbers together and save to history',
   'parameters': {'type': 'object',
    'properties': {'a': {'type': 'number', 'description': 'First number'},
     'b': {'type': 'number', 'description': 'Second number'}},
    'required': ['a', 'b']}}},
 {'type': 'function',
  'function': {'name': 'subtract',
   'description': 'Subtract b from a and save to history',
   'parameters': {'type': 'object',
    'properties': {'a': {'type': 'number', 'description': 'First number'},
     'b': {'type': 'number', 'description': 'Second number'}},
    'required': ['a', 'b']}}},
 {'type': 'function',
  'function': {'name': 'multiply',
   'description': 'Multiply two numbers and save to history',
   'parameters': {'type': 'object',
    'properties': {'a': {'type': 'number', 'description': 'First number'},
     'b': {'type': 'number', 'description': 'Second number'}},
    'required': ['a', 'b']}}},
 {'type': 'function',
  'functio

In [7]:
# user_input = "Find 5 minus 2"

# Request Mistral to perform the calculation
def generate_new_query(user_input, previous_result=None):
    
    response = mistral.chat.complete(
        model="mistral-small-latest",
        messages=[{
            "role": "system",
            "content": "You are a calculator assistant. Use the add or subtract functions to help the user with calculations."
        }, {
            "role": "user",
            "content": f"previous result is {previous_result} and user_input is {user_input}.  return operation and two relevant numbers to perform the calculation."
        }],
        tools=mistral_tools,
        tool_choice="auto"
    )
    return response.choices[0].message

In [8]:
# Define a series of calculations
calculations = [
    "add 10 and 5",
    "subtract 8 from 20",
    "multiply 7 and 6",
    "divide 100 into 4", #25
    "add another 5 to result" #30
]

async with sse_client(server_url) as (read, write):
    async with ClientSession(read, write) as session:
        await session.initialize()
        
        print("Performing calculations...\n")
        for operation in calculations:
            print(operation)
            previous_result = await session.call_tool("get_last_result")
            print("previous result", previous_result.content[0].text)
            message = generate_new_query(operation, previous_result)
            print(message)
            if message.tool_calls is not None:
                for tool_call in message.tool_calls:
                    # Extract function name
                    func_name = tool_call.function.name
                    
                    # Parse arguments (handle both string and dict formats)
                    if isinstance(tool_call.function.arguments, str):
                        args = json.loads(tool_call.function.arguments)
                    else:
                        args = tool_call.function.arguments
                    
                    # Execute the MCP tool
                    result = await session.call_tool(func_name, args)
                    print(f"Operation: {operation}, args: {args}, Result: {result.content[0].text}")
                    print("************************************")
            else:
                print("No tool calls found in the response.")

Performing calculations...

add 10 and 5
previous result Error calling tool 'get_last_result': No calculations performed yet
content='' tool_calls=[ToolCall(function=FunctionCall(name='add', arguments='{"a": 10, "b": 5}'), id='ew2mIRna2', type=None, index=0)] prefix=False role='assistant'
Operation: add 10 and 5, args: {'a': 10, 'b': 5}, Result: 15.0
************************************
subtract 8 from 20
previous result 15.0
content='' tool_calls=[ToolCall(function=FunctionCall(name='subtract', arguments='{"a": 20, "b": 8}'), id='i2RAAx91y', type=None, index=0)] prefix=False role='assistant'
Operation: subtract 8 from 20, args: {'a': 20, 'b': 8}, Result: 12.0
************************************
multiply 7 and 6
previous result 12.0
content='' tool_calls=[ToolCall(function=FunctionCall(name='multiply', arguments='{"a": 7, "b": 6}'), id='lZD1Pgq2N', type=None, index=0)] prefix=False role='assistant'
Operation: multiply 7 and 6, args: {'a': 7, 'b': 6}, Result: 42.0
*********************

## Access Calculation History Resource

Now let's retrieve the complete history of calculations using the history resource.

In [9]:
async with sse_client(server_url) as (read, write):
    async with ClientSession(read, write) as session:
        await session.initialize()
        
        # Read the history resource
        history_resource = await session.read_resource("calculation://history")
        history_data = json.loads(history_resource.contents[0].text)
        
        print(f"Total calculations in history: {history_data['count']}\n")
        print("Calculation History:")
        print("=" * 50)
        
        for calc in history_data['history']:
            print(f"ID: {calc['id']}")
            print(f"Time: {calc['timestamp']}")
            print(f"Operation: {calc['operation']}({calc['a']}, {calc['b']}) = {calc['result']}")
            print("-" * 30)

Total calculations in history: 5

Calculation History:
ID: 1
Time: 2025-07-06T11:25:56.969554
Operation: add(10.0, 5.0) = 15.0
------------------------------
ID: 2
Time: 2025-07-06T11:25:57.872961
Operation: subtract(20.0, 8.0) = 12.0
------------------------------
ID: 3
Time: 2025-07-06T11:25:58.420030
Operation: multiply(7.0, 6.0) = 42.0
------------------------------
ID: 4
Time: 2025-07-06T11:25:59.787841
Operation: divide(100.0, 4.0) = 25.0
------------------------------
ID: 5
Time: 2025-07-06T11:26:00.303221
Operation: add(25.0, 5.0) = 30.0
------------------------------


## Access Statistics Resource

Let's check the current statistics for our session.

In [10]:
async with sse_client(server_url) as (read, write):
    async with ClientSession(read, write) as session:
        await session.initialize()
        
        # Read statistics
        stats_resource = await session.read_resource("calculation://statistics")
        stats = json.loads(stats_resource.contents[0].text)
        
        print("Session Statistics:")
        print("=" * 30)
        print(f"Total Calculations: {stats['total_calculations']}")
        print(f"Last Result: {stats['last_result']}")
        print(f"Last Operation: {stats['last_operation']}")
        print(f"Session Start: {stats['session_start']}")

Session Statistics:
Total Calculations: 5
Last Result: 30.0
Last Operation: add
Session Start: 2025-07-06T11:25:50.659037


## Retrieve Specific Calculation by ID

Resources can have parameters. Let's retrieve a specific calculation using its ID.

In [11]:
# Retrieve calculation with ID 3
async with sse_client(server_url) as (read, write):
    async with ClientSession(read, write) as session:
        await session.initialize()
        
        # Read specific calculation
        calc_resource = await session.read_resource("calculation://history/3")
        calc = json.loads(calc_resource.contents[0].text)
        
        print("Calculation #3:")
        print(json.dumps(calc, indent=2))

Calculation #3:
{
  "id": 3,
  "timestamp": "2025-07-06T11:25:58.420030",
  "operation": "multiply",
  "a": 7.0,
  "b": 6.0,
  "result": 42.0
}


## Access Summary Resource

The summary resource provides aggregate information about all calculations.

In [12]:
async with sse_client(server_url) as (read, write):
    async with ClientSession(read, write) as session:
        await session.initialize()
        
        # Read summary
        summary_resource = await session.read_resource("calculation://summary")
        summary = json.loads(summary_resource.contents[0].text)
        
        print("Calculation Summary:")
        print("=" * 40)
        print(f"Total: {summary['total_calculations']} calculations")
        print(f"\nOperations breakdown:")
        for op, count in summary['operations_breakdown'].items():
            print(f"  - {op}: {count} times")
        
        print(f"\nResults statistics:")
        print(f"  - Min: {summary['results']['min']}")
        print(f"  - Max: {summary['results']['max']}")
        print(f"  - Average: {summary['results']['average']:.2f}")
        print(f"  - Last 5: {summary['results']['last_5_results']}")

Calculation Summary:
Total: 5 calculations

Operations breakdown:
  - add: 2 times
  - subtract: 1 times
  - multiply: 1 times
  - divide: 1 times

Results statistics:
  - Min: 12.0
  - Max: 42.0
  - Average: 24.80
  - Last 5: [15.0, 12.0, 42.0, 25.0, 30.0]


## Using Resources with Mistral AI

Now let's demonstrate how Mistral can use both tools and resources together. We'll prepare the tools for Mistral and create a more complex interaction.

In [13]:
# Prepare tools for Mistral
async with sse_client(server_url) as (read, write):
    async with ClientSession(read, write) as session:
        await session.initialize()
        tools_result = await session.list_tools()
        
        mistral_tools = []
        for tool in tools_result.tools:
            # Create a generic parameter schema
            parameters = {
                "type": "object",
                "properties": {},
                "required": []
            }
            
            # Add parameters based on tool name
            if tool.name in ["add", "subtract", "multiply", "divide"]:
                parameters["properties"] = {
                    "a": {"type": "number", "description": "First number"},
                    "b": {"type": "number", "description": "Second number"}
                }
                parameters["required"] = ["a", "b"]
            
            tool_def = {
                "type": "function",
                "function": {
                    "name": tool.name,
                    "description": tool.description,
                    "parameters": parameters
                }
            }
            mistral_tools.append(tool_def)
        
        print(f"Prepared {len(mistral_tools)} tools for Mistral")

Prepared 6 tools for Mistral


## Complex Calculation with History Check

Let's ask Mistral to perform a calculation and then tell us about the history.

In [14]:
# First, perform a calculation
user_input = "Calculate 123 times 45 for me"

response = mistral.chat.complete(
    model="mistral-small-latest",
    messages=[{
        "role": "system",
        "content": "You are a calculator assistant. Use the available functions to help with calculations."
    }, {
        "role": "user",
        "content": user_input
    }],
    tools=mistral_tools,
    tool_choice="auto"
)

# Execute the tool call
message = response.choices[0].message
if message.tool_calls:
    async with sse_client(server_url) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            
            for tool_call in message.tool_calls:
                func_name = tool_call.function.name
                args = json.loads(tool_call.function.arguments) if isinstance(tool_call.function.arguments, str) else tool_call.function.arguments
                
                result = await session.call_tool(func_name, args)
                print(f"Calculation result: {result.content[0].text}")

Calculation result: 5535.0


## Check Updated Statistics

After our Mistral-driven calculation, let's check how the statistics have changed.

In [ ]:
async with sse_client(server_url) as (read, write):
    async with ClientSession(read, write) as session:
        await session.initialize()
        
        # Get updated statistics
        stats_resource = await session.read_resource("calculation://statistics")
        stats = json.loads(stats_resource.contents[1].text)
        
        # Get the last result using the tool
        last_result = await session.call_tool("get_last_result", {})
        
        print("Updated Statistics:")
        print(f"Total Calculations: {stats['total_calculations']}")
        print(f"Last Operation: {stats['last_operation']}")
        print(f"Last Result (from tool): {last_result.content[0].text}")

Updated Statistics:
Total Calculations: 6
Last Operation: multiply
Last Result (from tool): 5535.0


## Interactive Example: Calculator with Memory

Let's create a function that combines tools and resources for a more interactive experience.

In [16]:
async def calculator_with_memory(expression: str):
    """Process a calculation and show relevant history"""
    print(f"Processing: {expression}")
    print("=" * 40)
    
    # Use Mistral to interpret and calculate
    response = mistral.chat.complete(
        model="mistral-small-latest",
        messages=[{
            "role": "system",
            "content": "You are a calculator. Parse the user's expression and use the appropriate function."
        }, {
            "role": "user",
            "content": expression
        }],
        tools=mistral_tools,
        tool_choice="auto"
    )
    
    # Execute the calculation
    message = response.choices[0].message
    if message.tool_calls:
        async with sse_client(server_url) as (read, write):
            async with ClientSession(read, write) as session:
                await session.initialize()
                
                for tool_call in message.tool_calls:
                    func_name = tool_call.function.name
                    args = json.loads(tool_call.function.arguments) if isinstance(tool_call.function.arguments, str) else tool_call.function.arguments
                    
                    # Perform calculation
                    result = await session.call_tool(func_name, args)
                    print(f"Result: {result.content[0].text}")
                    
                    # Get summary after calculation
                    summary_resource = await session.read_resource("calculation://summary")
                    summary = json.loads(summary_resource.contents[0].text)
                    
                    print(f"\nThis was calculation #{summary['total_calculations']}")
                    print(f"Session average: {summary['results']['average']:.2f}")

# Test the function
await calculator_with_memory("What is 88 divided by 11?")

Processing: What is 88 divided by 11?
Result: 8.0

This was calculation #7
Session average: 809.57


## Summary

In this notebook, we've demonstrated:

1. **MCP Resources**: How to access server state through resources
2. **State Persistence**: How calculations are stored and can be retrieved
3. **Resource Types**:
   - Simple resources (`calculation://history`)
   - Parameterized resources (`calculation://history/{id}`)
   - Aggregate resources (`calculation://summary`)
4. **Integration**: How to combine tools and resources for richer interactions
5. **Mistral AI Integration**: Using LLM to interpret requests and access calculator state

### Key Benefits of Resources:
- **State Management**: Maintain context across multiple tool calls
- **Data Access**: Provide read-only access to server data
- **Flexibility**: Support both simple and parameterized queries
- **Integration**: Work seamlessly with tools for complete functionality

This pattern can be extended to more complex applications where maintaining state and providing data access is crucial.